# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
weather_data_file = "../output_data/cities.csv"
weather_df = pd.read_csv(weather_data_file, encoding = "UTF-8")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,78.13,70,25,5.79,PF,1612839769
1,Esperance,-33.8667,121.9000,71.60,53,75,8.05,AU,1612839491
2,Port Alfred,-33.5906,26.8910,66.99,89,9,4.74,ZA,1612839769
3,Kapaa,22.0752,-159.3190,69.80,83,75,18.41,US,1612839769
4,Namatanai,-3.6667,152.4333,84.49,77,55,8.61,PG,1612839769


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"]

In [11]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=30, point_radius=3.0)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Drop any rows with null values
weather_df.dropna(inplace = True) 
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,78.13,70,25,5.79,PF,1612839769
1,Esperance,-33.8667,121.9000,71.60,53,75,8.05,AU,1612839491
2,Port Alfred,-33.5906,26.8910,66.99,89,9,4.74,ZA,1612839769
3,Kapaa,22.0752,-159.3190,69.80,83,75,18.41,US,1612839769
4,Namatanai,-3.6667,152.4333,84.49,77,55,8.61,PG,1612839769


In [43]:
# Create new DataFrame with cities that meet specified weather condition parameters
ideal_weather_df = weather_df[(weather_df["Max Temp"]<75) & (weather_df['Max Temp']>68) & 
                              (weather_df['Humidity']<75) & (weather_df['Wind Speed']>10)]
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,Geraldton,-28.7667,114.6000,73.40,56,40,14.97,AU,1612839774
101,Eyl,7.9803,49.8164,71.29,72,16,16.49,SO,1612839545
185,Rājula,21.0500,71.4333,68.14,28,7,10.16,IN,1612839783
189,Ahipara,-35.1667,173.1667,71.76,70,99,11.74,NZ,1612839783
192,Umluj,25.0213,37.2685,71.87,36,43,11.36,SA,1612839784
239,Codrington,-38.2667,141.9667,70.03,51,0,12.26,AU,1612839680
295,Mount Gambier,-37.8333,140.7667,71.60,43,0,11.50,AU,1612839791
357,Bandarbeyla,9.4942,50.8122,74.61,73,61,20.76,SO,1612839797
423,Bulancak,40.9372,38.2291,73.40,20,75,11.50,TR,1612839781
455,Görele,41.0308,39.0031,73.40,27,75,13.80,TR,1612839802


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
hotel_df = ideal_weather_df[['Country', 'City', 'Lat', 'Lng']]
hotel_df['Hotel Name']= ''
hotel_df

C:\Users\bisma\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Country,City,Lat,Lng,Hotel Name
61,AU,Geraldton,-28.7667,114.6000,
101,SO,Eyl,7.9803,49.8164,
185,IN,Rājula,21.0500,71.4333,
189,NZ,Ahipara,-35.1667,173.1667,
192,SA,Umluj,25.0213,37.2685,
239,AU,Codrington,-38.2667,141.9667,
295,AU,Mount Gambier,-37.8333,140.7667,
357,SO,Bandarbeyla,9.4942,50.8122,
423,TR,Bulancak,40.9372,38.2291,
455,TR,Görele,41.0308,39.0031,


In [48]:
# find the closest hotel to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat and Lng coordinates from df
    lat = row ['Lat']
    lng = row ['Lng']
    city = row ['City']

    # add keyword to params dict
    params['location'] = (f'{lat},{lng}')

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("-------------------------")

Closest hotel in Geraldton is Broadwater Mariner Resort.
-------------------------


C:\Users\bisma\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel in Eyl is Amiin Hotel.
-------------------------
Closest hotel in Rājula is Lion Lords Inn Rajula.
-------------------------
Closest hotel in Ahipara is Ahipara Holiday Park.
-------------------------
Closest hotel in Umluj is Moon light Furnished Units.
-------------------------
Closest hotel in Codrington is Codrington Gardens Bed And Breakfast.
-------------------------
Closest hotel in Mount Gambier is Commodore on the Park.
-------------------------
Closest hotel in Bandarbeyla is JABIR HOTEL.
-------------------------
Closest hotel in Bulancak is Ada Loft Aparts.
-------------------------
Closest hotel in Görele is Görele Öğretmenevi.
-------------------------
Closest hotel in Sur is Sur Plaza Hotel.
-------------------------


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))